# DSGE and State-Space Modeling with Julia
*2017 Reserve Bank Workshops*

** *Erica Moszkowski* ** *(@emoszkowski)*, <br/>
***Federal Reserve Bank of New York*** *(@FRBNY-DSGE)*

*3/10/2017 - 3/13/2017*

## Disclaimer

This talk reflects the experience of the speaker with Julia and MATLAB and does
not represent an endorsement by the Federal Reserve Bank of New York or the
Federal Reserve System of any particular product or service. The views
expressed in this talk are those of the speaker and do not necessarily reflect
the position of the Federal Reserve Bank of New York or the Federal Reserve System.

## Outline

- Quick review of DSGEs
- DSGE.jl: from MATLAB to Julia
  - How?
  - Why? 
- Code design approaches in DSGE.jl
- More useful tools: StateSpaceRoutines.jl
- Exercise

## Collaborators

- FRBNY-DSGE team: 
  - Marco Del Negro, Marc Giannoni, Abhi Gupta, Pearl Li, Sara Shahanaghi, Micah Smith
- QuantEcon collaborators:
  - Zac Cranko, Spencer Lyon, John Stachurski, Pablo Winant

## The DSGE model 

A DSGE model is a system of dynamic equations: 

<center>$ \Gamma_0(\theta) s_{t} = \Gamma_1(\theta) s_{t-1} + \Gamma_C(\theta) + \Psi(\theta) \epsilon_{t} + \Pi(\theta) \eta_{t} $ &emsp; $\epsilon_t \sim N(0,Q(\theta))$</center> 

where
- $s_t$ is a vector of states at time $t$ (capital, inflation, ...)
  - includes forecasts of future states
- $\epsilon_t$ is a vector of shocks (productivity, financial,...)
- $\eta_t$ is a vector of forecast errors (under rational expectations in the current version of the FRBNY DSGE model)
- $\theta$ is a vector of parameters describing preferences and technology (depreciation rate, ...)

## The DSGE model (ctd) 

The model solution (given by `gensys`) is the *transition equation*:

<center>$ s_{t}  = T(\theta) s_{t-1} + R(\theta) \epsilon_{t} + C(\theta) $ &emsp; &emsp; &emsp; (1)</center> 

We add a *measurement equation* to map states to observed data ($y_t$): 

<center>$ y_{t}  = Z(\theta) s_{t} + D(\theta)$ &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; (2) </center>

Equations (1) and (2) form a state-space system.

## The DSGE model (ctd)

**A DSGE model is a *mapping* between $\theta$ and $\Gamma_0, \Gamma_1, \Gamma_C, \Psi, \Pi$, $Q, Z$, and $D$.**

- This mapping is needed to compute the likelihood $p(y|\theta$) from the state-space model. 
  - The likelihood is the probability of observing the data given a set of parameters.
  - `DSGE.jl` centers around a "model object" that stores this mapping. 

- We adopt a Bayesian approach: $\theta$ is a random variable, and we impose a prior $p(\theta)$ on it.
   - The `DSGE.jl` model object also includes information on this prior.

- Once we know $\theta$, we know everything -> $\theta$ is the key variable of interest

## Estimating the parameter values

- Goal: conduct inference about $\theta$ and $s_t$  (obtain draws from the posterior distribution $p(\theta|y) \propto p(y|\theta)p(\theta))$

- How? Sample from posterior distribution using `metropolis_hastings` MCMC method.
  - This is the most computationally expensive part of estimation (takes ~10 hrs)

In [ ]:
for i = 1:120000   # metropolis_hastings *very* computationally intensive

    draw θ_i
    solve(model, θ_i)                       # `gensys`
    loglike = likelihood(model, data, θ_i)  # `kalman_filter` => p(data | θ_i)
    logpost = loglike + logprior(θ_i)       # compute p(θ_i | data)
    
end

## Forecasting states and observables 

Iterate state-space system forward from $T$ (end of sample) to forecast future values of states and observables:

$$ s_{T+1}  = T(\theta) s_{T} + R(\theta) \epsilon_{T+1} + C(\theta) $$
$$ y_{T+1}  = Z(\theta) s_{T+1} + D(\theta) $$ <br>

We can do this for $1,2,\dots,h$ periods ahead, iterating forward one period at a time.

## `DSGE.jl`

- Julia package that facilitates the solution and Bayesian estimation of DSGE models
<br>
- Users specify their own models and apply our implementation of workhorse procedures
<br>
- Main sections:
    1. Prepare data 
      - Integrates with FredData.jl to pull from Federal Reserve Economic Data (FRED)
    2. Solve model and estimate time-invariant parameters 
    3. Forecast macro variables (e.g. GDP growth, inflation) 
    4. Plot (not yet released)
    5. ...

## `using DSGE`

In [ ]:
using DSGE

# construct a model object
m = Model990()

# estimate as of 2015-Q3 using the default data vintage from 2015 Nov 27
m <= Setting(:data_vintage, "151127")
m <= Setting(:date_mainsample_end, quartertodate("2015-Q3"))

# loads data from FRED and user-supplied CSV files
df = load_data(m)   

# reoptimize parameter vector, compute Hessian at mode, and full posterior
# parameter sampling
estimate(m, df)

# produce LaTeX tables of parameter moments
compute_moments(m)

## From MATLAB to Julia: The Story

**~10 years ago** &emsp;  &emsp;  DSGE team begins! 

**September 2014** &emsp;   MATLAB codebase made public (in .zip) on *Liberty Street Economics* 

**Spring 2015** &emsp; &emsp; &emsp;  QuantEcon approaches DSGE team  

<ul style="margin-top:0px;"><li>For QuantEcon</li>
  <ul><li>Add production DSGE model from central bank to collection of high-quality open-source code for economics</li></ul></ul>
  
<ul style="margin-top:0px;"><li>For DSGE team</li>
  <ul><li>Improve performance sufficiently to experiment with new techniques (e.g. nonlinear models)</li>
  <li>Redesign codebase from bottom up</li>
  <li>Make code free!</li></ul></ul>
  
**December 2015** &emsp;   Release of DSGE.jl (estimation step)

**June 22, 2016** &emsp; &emsp; Release of data step (v0.1.3) 

**March/April, 2017** &nbsp; Release of forecast step (v0.2.0) <br>
        &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &emsp; &nbsp; Release of StateSpaceRoutines.jl <br>

## Why Julia?  

**Initial reasoning**
  - A bit of a push from John and QuantEcon (thank you!)
  - Some Julia features make it easier for MATLAB users to pick up (1-indexing, for example)

**With some reflection**

A number of Julia <font color="blue">features</font> lend themselves to DSGE.jl <font color="green">implementation</font>
  - Central <font color="blue">type system</font> provides a natural way to <font color="green">organize</font> and <font color="green">simplify</font> codebase
  - <font color="blue">Method dispatch</font> allows us to write more <font color="green">general</font> code without sacrificing speed
  - Rely on the <font color="blue">compiler</font> to boost <font color="green">performance</font>

*Keep in mind:* DSGE.jl is not a direct port! Even so...some motivating stats:

## Types + dispatch = simpler code

<br>

| Language             | Lines of code (hundreds) |
| -------------------- | :----------------------: |
| Matlab               | 63                       |
| Julia                | 37                       |

<center> Lines of code used in estimation step. *(Including redesign)* 

## <font color="green">Performance</font> improvements

No substantial redesign:

| Test                  | MATLAB (14a) | Julia (0.4.0)  |
| --------------------  | :----------: | :------------: |
| `gensys`              | 1.00         | 5.88           |
| `solve`               | 1.00         | 11.11          |
| `kalman_filter`       | 1.00         | 1.33           |
| `posterior`           | 1.00         | 3.85           |
| `csminwel`            | 1.00         | 3.03           |
| `hessian`             | 1.00         | 4.35           |


Substantial redesign:

| Test                  | MATLAB (14a) | Julia (0.4.0)  |
| --------------------  | :----------: | :------------: |
| `metropolis_hastings` | 1.00         | 9.09           |

<center>Benchmark speedup relative to MATLAB (larger is better) </center>

<center>*On an Intel® Xeon® E5-2697 v2 2.70GHz CPU running GNU/Linux*</center>

## DSGE.jl design: types all the way down

- In Julia, it's natural to take a *type-oriented* approach to an economic model 


- Types allow for intuitive expression of economic objects (e.g. models, parameters)
   - Encapsulate all features of that object in a single data structure
   - Seamless dispatch to support economic intuition
   
   
  
- Applications in `DSGE.jl`: 
   - `AbstractModel`
   - `Parameters`
   - `Settings`

In [1]:
using DSGE, Distributions

    all(Any, Lazy.List) at /Users/ericamoszkowski/.julia/v0.4/Lazy/src/liblazy.jl:187
is ambiguous with: 
    all(Base.Predicate, Any) at reduce.jl:369.
To fix, define 
    all(Base.Predicate, Lazy.List)
before the new definition.
    all(Any, Lazy.List) at /Users/ericamoszkowski/.julia/v0.4/Lazy/src/liblazy.jl:187
is ambiguous with: 
    all(Base.IdFun, Any) at reduce.jl:370.
To fix, define 
    all(Base.IdFun, Lazy.List)
before the new definition.
    all(Any, Lazy.List) at /Users/ericamoszkowski/.julia/v0.4/Lazy/src/liblazy.jl:187
is ambiguous with: 
    all(AbstractArray, Any) at reducedim.jl:264.
To fix, define 
    all(AbstractArray, Lazy.List)
before the new definition.
    any(Any, Lazy.List) at /Users/ericamoszkowski/.julia/v0.4/Lazy/src/liblazy.jl:184
is ambiguous with: 
    any(Base.Predicate, Any) at reduce.jl:362.
To fix, define 
    any(Base.Predicate, Lazy.List)
before the new definition.
    any(Any, Lazy.List) at /Users/ericamoszkowski/.julia/v0.4/Lazy/src/liblazy.jl:1

## Case study #1: `Parameter` types

Recall that DSGE parameters (θ) aren't just values. They have a number of features:

In [2]:
type Parameter{T<:Number}
    key::Symbol
    value::T                        # parameter value 
    valuebounds::Tuple{T,T}         # bounds of parameter value
    prior::Distribution             # prior distribution
    fixed::Bool                     # is this parameter fixed at some value?
    description::AbstractString
    tex_label::AbstractString       # LaTeX label for printing
end

α = Parameter(:α, 0.1596, (1e-5, 0.999), Normal(0.30, 0.05),  
              false, "α: Capital share", "\\alpha")

Parameter{Float64}(:α,0.1596,(1.0e-5,0.999),Distributions.Normal{Float64}(μ=0.3, σ=0.05),false,"α: Capital share","\\alpha")

## Case study #1 (ctd):  multiple dispatch supports economic intuition

We should be able to have α + 1. 

In Julia, operators are just methods. That means we can add a method to the plus sign:

In [3]:
Base.(:+)(p::Parameter, x::Number) = p.value + x
α + 1.

1.1596

We should also be able to add parameters on the right: 1 + α

In [4]:
Base.(:+)(x::Number, p::Parameter) = p + x
1. + α   # has meaning! 

1.1596

The ease with which we can do this is fairly unique to Julia, or at least languages with multiple dispatch.

## Case study #1 (ctd)  

Why is parameter arithmetic interesting? 

Because we can define equilibrium conditions using parameter algebra! 

In [ ]:
y_t = Φ*(α*k_t + (1-α)*L_t)   # Log-linearized production function 

This is a only *slight* simplification of how a user would implement their equilibrium conditions and measurement equation in DSGE.jl

## Exercise 1

Using the following information,  evaluate the production function `y_t = Φ*(α*k_t + (1-α)*L_t)`.

```julia
type Parameter{T<:Number}
    key::Symbol
    value::T                        # parameter value 
    prior::Distribution             # prior distribution
    fixed::Bool                     # is this parameter fixed at some value?
end
Base.(:+)(p::Parameter, x::Number) = p.value + x
Base.(:+)(x::Number, p::Parameter) = p + x
```


Fix α to 0.33. Let Φ have a Normal prior with mean 1.25 and standard deviation 0.1, and initialize its value to 1.2. Let k_t = 1 and L_t = 1.5.

Hints:
1. `α` and `Φ` need not be unicode, though you are welcome to do so if you like.
2. Make sure to use the Distributions package (`using Distributions`)

## Exercise 1 Solution


In [5]:
using Distributions

# Define a Parameter type
type MyParameter{T<:Number}
    key::Symbol
    value::T                        # parameter value 
    prior::Distribution             # prior distribution
    fixed::Bool                     # is this parameter fixed at some value?
end

In [6]:
# Add methods to * and -
for op in [:(Base.(:-)), :(Base.(:*))]
    @eval ($op)(p::MyParameter, x::Number) = ($op)(p.value, x)
    @eval ($op)(x::Number, p::MyParameter) = ($op)(x, p.value)
end

# Initialize parameter objects
α  = MyParameter(:α, 0.33, Normal(0.33, 0.0001), true)
Φ  = MyParameter(:Φ, 1.2, Normal(1.25, 0.1), false)

# Initialize states
k_t = 1.
L_t = 1.5

y_t = Φ*(α*k_t + (1-α)*L_t)

1.6019999999999999

## Case study #2a: Dealing with computational settings

- "Computational settings" = flags or numbers without economic intiution
- There are *many* of these in a production DSGE model like ours:
  - Can we read in precomputed modal parameters, or should we optimize?
  - How many draws should take from the posterior distribution in Metropolis-Hastings?
  - Where do we store input/output files?
- Don't want to have to track them down somewhere in the source code to change their values
- Want to deal with all settings uniformly

In [ ]:
type Setting{T}
    key::Symbol                  # name of setting
    value::T                     # whatever the setting is
    print::Bool                  # add this setting to file string?
    code::AbstractString         # what gets printed to the print
    description::AbstractString  # description of what the setting is for
end

- Store a `Dict{Setting}` within the model object - makes it easy to look settings up and pass them around

## Exercise 2

Imagine you have a model you want to estimate and forecast. You have data as of November 27,  2015 and you want to forecast the model 60 periods ahead. 

Suppose you already have a model object set up. You want to add a dictionary of `Setting`s to it so that it will be easy to change the data vintage and horizon later on. 

Create a `Dict{Symbol,Setting}` with 2 `Settings` in it: one representing the data vintage and one representing the number of periods to forecast. Use the following definition of a `Setting`:
```julia
type Setting{T}
    key::Symbol                  # name of setting
    value::T                     # whatever the setting is
    print::Bool                  # add this setting to file string?
    code::AbstractString         # what gets printed to the print
    description::AbstractString  # description of what the setting is for
end
```

## Exercise 2 Solution

In [7]:
# Copy Setting definition
type MySetting{T}
    key::Symbol                  # name of setting
    value::T                     # whatever the setting is
    print::Bool                  # add this setting to file string?
    code::AbstractString         # what gets printed to the print
    description::AbstractString  # description of what the setting is for
end
# Setup (within model constructor)
settings = Dict{Symbol,MySetting}()

Dict{Symbol,MySetting{T}} with 0 entries

In [8]:
# Construct settings
settings[:vint]  = MySetting(:data_vintage, "151127", 
                       true, "vint", "input data vintage")
settings[:n_forecast_periods] = MySetting(:n_forecast_periods, 60, 
                       true, "hrzn", "number of periods to forecast")

settings

Dict{Symbol,MySetting{T}} with 2 entries:
  :vint               => MySetting{ASCIIString}(:data_vintage,"151127",true,"vi…
  :n_forecast_periods => MySetting{Int64}(:n_forecast_periods,60,true,"hrzn","n…

## Case study #2b:  using `Settings` to prevent file collisions

- **Problem:**
  - We create a lot of output files (particularly in `forecast`)
  - Don't want to overwrite files when using different `Setting`s

- **Solution:**
  - `print` and `code` fields of `Setting` object are used to construct a file string 

In [ ]:
to_filestring(s::Setting) = "$(s.code)=$(s.value)"

## Case study #2b (ctd):  using `Settings` to prevent file collisions

Let's use the same dictionary of settings from Exercise 2:

In [ ]:
settings = Dict{Symbol,Setting}()
settings[:vint]  = Setting(:data_vintage, "151127", 
                       true, "vint", "input data vintage")
settings[:n_forecast_periods] = Setting(:n_forecast_periods, 60, 
                       true, "hrzn", "number of periods to forecast")

In DSGE.jl, when we construct a filename, we run the following code and append the result to a base filename: 

In [9]:
filestrings = Vector{ASCIIString}()
for (skey, sval) in settings
    if sval.print
        push!(filestrings, "$(sval.code)=$(sval.value)") 
    end
end

join(filestrings,'_')                           # join all filestrings together

"vint=151127_hrzn=60"

## Case study #2b (ctd): using `Settings` to prevent file collisions

- Approach to `Setting`s and filestrings is not unique to Julia


- These are **very common** challenges and everyone has their own way of dealing with them


- This is our approach - we like it, but you can take it or leave it. 

## `DSGE.jl`: Ongoing and future work

### Ongoing
- Plotting


### Future directions
- Nonlinear solution methods
- New optimization methods
  - E.g. `csminwel` => simulated annealing?
- New estimation methods
  - E.g. `metropolis_hastings` => Sequential Monte Carlo

## `StateSpaceRoutines.jl`

<center>$ s_{t}  = T_i(\theta) s_{t-1} + R_i(\theta) \epsilon_{t} + C_i(\theta) $  </center> 
<center>$ y_{t}  = Z_i(\theta) s_{t} + D_i(\theta)$  </center>

- State space routines are not only used to estimate and forecast DSGE models 
- Don't want others to waste time implementing these routines again

`StateSpaceRoutines.jl` provides model-agnostic, optionally time-varying, state-space routines
  - Inputs and outputs are just matrices!
  - `DSGE.jl` uses `StateSpaceRoutines.jl` as a dependency.
  

## `StateSpaceRoutines.jl`: Available routines

- Kalman filter (`kalman_filter`): computes log-likelihood and filtered states $s_{t|1:t}$ for $t \in 1:T$

- Kalman smoothers: compute $E [s_{t|} | y_{1:T} ]$ and $Var [s_t | y_{1:T} ]$
  + `hamilton_smoother`: James Hamilton, [_Time Series Analysis_](https://www.amazon.com/Time-Analysis-James-Douglas-Hamilton/dp/0691042896) (1994)
  + `koopman_smoother`: S.J. Koopman, ["Disturbance Smoother for State Space Models"](https://www.jstor.org/stable/2336762) (_Biometrika_, 1993)

- Simulation smoothers: draw from $p(s_{T}| \theta, y_{1:T})$
  + `carter_kohn_smoother`: C.K. Carter and R. Kohn, ["On Gibbs Sampling for State Space Models"](https://www.jstor.org/stable/2337125) (_Biometrika_, 1994)
  + `durbin_koopman_smoother`: J. Durbin and S.J. Koopman, ["A Simple and Efficient Simulation Smoother for State Space Time Series Analysis"](https://www.jstor.org/stable/4140605) (_Biometrika_, 2002)


- Future work
   - We have a Bayesian VAR package also in the works! Keep an eye on *Liberty Street Economics* or our GitHub page

## Example state-space model: Univariate MA(1) process

$x_t = μ + ε_t +  θ₁*ε_{t-1}$ &emsp; $ε \sim N(0, I)$

State-space representation:
z_t = [ε_t, ε_{t-1}]

Transition equation: <br/>
[ $ε_{t}$      =   [0 0  [ $ε_{t-1}$  + [1  [$ε_{t}$] <br/>
  $ε_{t-1}$ ]       1 0]   $ε_{t-2}$]    0] <br/>

TTT = [0 0 ;
       1 0]
RRR = [1  ;  CCC = [0
       0]           0]
       
**Measurement equation:**
$y_t = x_t$

ZZ = [1 θ]
DD = [μ]


# Exercise 3: Univariate MA(1)

#### Model

\begin{equation}
x_t = \mu + \epsilon_t +  \theta *\epsilon_{t-1} \hspace{1em}  \epsilon_t \sim N(0, I) 
\end{equation}

#### State Space Representation 

\begin{equation}
z_t =  
\begin{bmatrix}
\epsilon_t \\
\epsilon_{t-1} 
\end{bmatrix}
\end{equation}

*Transition equation*

\begin{equation}
\begin{bmatrix}
\epsilon_t \\
\epsilon_{t-1} 
\end{bmatrix}
= 
\begin{bmatrix}
0 & 0  \\
1 & 0 \\
\end{bmatrix}
+ 
\begin{bmatrix}
1  \\
0 \\
\end{bmatrix}
[\epsilon_t]
\end{equation}

\begin{equation}
 TTT =
 \begin{bmatrix} 
 0 & 0 \\
1  & 0
 \end{bmatrix}, 
 RRR =
 \begin{bmatrix} 
  1 \\
  0
  \end{bmatrix}, 
  CCC = 
   \begin{bmatrix} 
  0 \\
  0
  \end{bmatrix}
 \end{equation}

*Measurement equation*
\begin{equation}
 y_t = x_t, \hspace{1em}
ZZ  = \begin{bmatrix} 
1 & \theta 
\end{bmatrix}, \hspace{1em}
DD = [\mu]
\end{equation}

## Exercise 3 solution

In [8]:
using StateSpaceRoutines

theta = 0.9; mu = 0.0

# initial state vector and covariance matrix
z0_init = zeros(2)
P0_init = eye(2)

# Transition matrices
TTT = [0.0   0.0;
       1.0   0.0]
RRR = [1.0 0.0]'
CCC = zeros(2)
    
# Measurement equation
QQ = eye(1)
ZZ = [1.0 theta]
DD = [mu]
EE = zeros(1,1)

data = readcsv("data_ma1.csv")
log_likelihood, pred, vpred, filt, vfilt, yprederror, ystdprederror, rmse, rmsd, z0, P0 = 
 kalman_filter(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, z0_init, P0_init, n_presample_periods = 1);

## Lessons We've Learned 

**We love open source (and Julia)!**
- Open-source languages and packages reduce costs of writing code and make it easier to share
- Julia is high-performance and high-productivity

**Challenges to be aware of**
- New language: frequent updates. This will slow down when v1.0 comes out (hopefully this year)
- Sparse StackOverflow activity

**We are sure there are better ways to do just about anything in DSGE.jl and StateSpaceRoutines.jl - we welcome your pull requests and thoughts!**

## Thank you!  

<br/ >
https://github.com/FRBNY-DSGE/DSGE.jl/<br/>
https://github.com/FRBNY-DSGE/StateSpaceRoutines.jl/<br/>

@emoszkowski  &emsp;  erica.moszkowski@gmail.com, Erica.Moszkowski@ny.frb.org <br/>
@pearlzli     &emsp; &nbsp; &emsp; &emsp; pearlzli16@gmail.com, Pearl.Li@ny.frb.org  <br/>
@FRBNY-DSGE   &nbsp; ny.dsge.ny@ny.frb.org